Python script for arming and disarming a plane


In [ ]:
import time
from pymavlink import mavutil

mavutil.set_dialect("ardupilotmega")

autopilot = mavutil.mavlink_connection('udpin:127.0.0.1:14551')

msg = None

# wait for autopilot connection
while msg is None:
        msg = autopilot.recv_msg()

print(msg)

# The values of these heartbeat fields is not really important here
# I just used the same numbers that QGC uses
# It is standard practice for any system communicating via mavlink emit the HEARTBEAT message at 1Hz! Your autopilot may not behave the way you want otherwise!
autopilot.mav.heartbeat_send(
6, # type
8, # autopilot
192, # base_mode
0, # custom_mode
4, # system_status
3  # mavlink_version
)

autopilot.mav.command_long_send(
1, # autopilot system id
1, # autopilot component id
400, # command id, ARM/DISARM
0, # confirmation
1, # arm!
0,0,0,0,0,0 # unused parameters for this command
)

time.sleep(2)

autopilot.mav.command_long_send(
1, # autopilot system id
1, # autopilot component id
400, # command id, ARM/DISARM
0, # confirmation
0, # disarm!
0,0,0,0,0,0 # unused parameters for this command
)

Function for setting waypoints 

In [ ]:
#mavwp set-up
wp = mavwp.MAVWPLoader()

def set_waypoint_cmd_mavwp(lat,lon,alt):
    wp.add(mavutil.mavlink.MAVLink_mission_item_message(
        1,#sys id
        1,#component id
        0,#seq wp id
        0,#frame id
        16,#mav_cmd 
        0,#current, true/false
        0,#auto continue
        0,0,0,0,#params 1-4: hold time(s),acceptable radius(m), pass/orbit,yaw angle
        lat,lon,alt) #lat/lon/alt
    )

In [ ]:
print "#of waypoints=",wp.count(),"\n"
    print "waypoint is=",wp.wp(0),"\n"
    autopilot.waypoint_clear_all_send()
    autopilot.waypoint_count_send(wp.count())
    msg = autopilot.recv_match(type=['MISSION_REQUEST'],blocking=True)
    print msg
    seq=0
    autopilot.mav.send(wp.wp(msg.seq))

https://www.colorado.edu/recuv/2015/05/25/mavlink-protocol-waypoints

Need to clone this first 

https://github.com/ArduPilot/pymavlink


Entering waypoints manually using python terminal

In [ ]:
from pymavlink import mavutil, mavwp

print("Enter the no. of waypoints: ")
N=int(input())
lat=list()
lon=list()
alt=list()

for i in range(N):
    print("Enter lat,long,alt")
    lat.append(float(input()))
    lon.append(float(input()))
    alt.append(float(input()))
    
master = mavutil.mavlink_connection('udp:localhost:14551')  
master.wait_heartbeat(blocking=True)                                       
wp = mavwp.MAVWPLoader()                                                    
seq = 1
frame = mavutil.mavlink.MAV_FRAME_GLOBAL_RELATIVE_ALT
radius = 10
for i in range(N):                  
            wp.add(mavutil.mavlink.MAVLink_mission_item_message(master.target_system,
                         master.target_component,
                         seq,
                         frame,
                         mavutil.mavlink.MAV_CMD_NAV_WAYPOINT,
                         0, 0, 0, radius, 0, 0,
                         lat[i],lon[i],alt[i]))
            seq += 1                                                                       

master.waypoint_clear_all_send()                                     
master.waypoint_count_send(wp.count())                          

for i in range(wp.count()):
            msg = master.recv_match(type=['MISSION_REQUEST'],blocking=True)             
            master.mav.send(wp.wp(msg.seq))                                                                      
            print('Sending waypoint {0}'.format(msg.seq))                                                     